In [1]:
import sys
sys.path.insert(0,'/Users/august/Coding/topic-modelling')

In [2]:
from sklearn.datasets import fetch_20newsgroups
from topic_modelling.data_loader import DataLoader
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd

categories = ['rec.sport.baseball', 'talk.religion.misc',
              'comp.graphics', 'sci.space', 'talk.politics.guns']
n_clusters = 5
categories = categories[:n_clusters]

newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', 
                                      categories=categories,  remove=('headers', 'footers', 'quotes'))
data_df = pd.DataFrame(newsgroups_train, columns=["data", "target"])
test_data_df = pd.DataFrame(newsgroups_test, columns=["data", "target"])

In [3]:
dl = DataLoader()
corpus = dl.load_df(data_df)

Processed data


In [4]:
from topic_modelling.kmeans import KMeansCluster
cluster = KMeansCluster(corpus["data"], TFIDF_method="native", num_clusters=n_clusters)
doc_clusters, cluster_assignments = cluster.cluster_documents(verbosity=False)

100%|██████████| 2623/2623 [00:00<00:00, 43117.38it/s]


fitted TF-IDF matrix


In [5]:
dl_test = DataLoader()
test_corpus = dl_test.load_df(test_data_df)

Processed data


In [6]:
cluster.get_best_words()

['color (0.01)', 'appreci (0.01)', 'format (0.01)', 'window (0.01)', 'object (0.01)', 'code (0.01)', 'gif (0.01)', 'anybodi (0.01)', 'card (0.01)', 'advanc (0.01)']
['player (0.05)', 'pitch (0.05)', 'hit (0.05)', 'win (0.04)', 'basebal (0.04)', 'score (0.03)', 'brave (0.03)', 'fan (0.03)', 'leagu (0.03)', 'season (0.03)']
['nasa (0.05)', 'orbit (0.04)', 'launch (0.04)', 'moon (0.03)', 'cost (0.03)', 'shuttl (0.03)', 'develop (0.03)', 'data (0.03)', 'satellit (0.02)', 'fund (0.02)']
['christian (0.15)', 'god (0.14)', 'jesus (0.11)', 'bibl (0.05)', 'christ (0.05)', 'mormon (0.03)', 'love (0.03)', 'church (0.03)', 'faith (0.03)', 'moral (0.03)']
['weapon (0.05)', 'fire (0.04)', 'fbi (0.04)', 'firearm (0.03)', 'crime (0.03)', 'crimin (0.03)', 'polic (0.02)', 'koresh (0.02)', 'child (0.02)', 'batf (0.02)']


In [7]:
categories

['rec.sport.baseball',
 'talk.religion.misc',
 'comp.graphics',
 'sci.space',
 'talk.politics.guns']

In [14]:
def rearrange_predictions(cluster: float):
    remap = [2, 0, 3, 1, 4]
    return remap[cluster]

In [15]:
test_matrix = np.zeros((len(test_corpus), cluster.matrix.shape[1]))

for i, row in enumerate(test_corpus["data"]):
    test_matrix[i] = cluster.vectorizer.fit_new_sentence(row)

In [16]:
n = NearestNeighbors(n_neighbors=1, metric="cosine")  
n.fit(cluster.cluster_centers)  

_, neigh_index = n.kneighbors(test_matrix)

In [17]:
preds = np.vectorize(rearrange_predictions)(neigh_index)
train_assign_labels = np.vectorize(rearrange_predictions)(cluster_assignments)

In [18]:
# test-data - osedd data
from sklearn.metrics import accuracy_score
accuracy_score(test_corpus["target"], preds)

0.09885714285714285

In [19]:
# träaningsdata - sedd data
accuracy_score(corpus["target"], train_assign_labels)

0.12466641250476554